# Main Causework UP2089158 UP2060325

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

## Settings & Load data

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # tf will show error messages only (reduce verbosity)
sns.set_style('white')

(X_train, y_train), (X_test, y_test) = cifar100.load_data()
print(X_train.shape, X_test.shape)

n_labels = len(np.unique(y_train))
n_labels

## Workflow for data

In [ ]:
pointer = 60 # number of image in the dataset (remember, numbering starts from 0!)

print(f"array pointer = {pointer}")
print(f"x_train[{pointer}] shape: {X_train[pointer].shape}")
print(f"label: {y_train[pointer]}")

plt.imshow(X_train[pointer],cmap='Accent')
plt.show()

## Data preparation

In [ ]:
def check_images(dataset, dataset_name):
    """
    Checks images for:
    * being an array
    * shape (28x28)
    * colour channel values
    * NaN values
    """
    invalid_count = 0  # Counter for invalid images
    valid_count = 0     # Counter for valid images

    for idx, image in enumerate(dataset):
        # Check if the image is a NumPy array
        if not isinstance(image, np.ndarray):
            print(f"{dataset_name} - Index {idx}: Not a valid image array")
            invalid_count += 1
            continue

        # Check shape (should be 28x28)
        if image.shape != (28, 28):
            print(f"{dataset_name} - Index {idx}: Incorrect shape {image.shape}")
            invalid_count += 1
            continue

        # Check if values are within expected range (0-255 for grayscale images)
        if not (image.dtype == np.uint8 and image.min() >= 0 and image.max() <= 255):
            print(f"{dataset_name} - Index {idx}: Invalid pixel values (Min: {image.min()}, Max: {image.max()})")
            invalid_count += 1
            continue

        # Check for NaN values
        if np.isnan(image).any():
            print(f"{dataset_name} - Index {idx}: Contains NaN values")
            invalid_count += 1
            continue

        valid_count += 1

    print(f"\n{dataset_name}: {valid_count} valid images, {invalid_count} invalid images")

    # Run checks on both datasets
print("Checking Images...\n")
check_images(X_train, "Train")
check_images(X_test, "Test")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
                                    X_train,
                                    y_train,
                                    test_size=0.2,
                                    random_state=0
                                    )

print("* Train set:", X_train.shape, y_train.shape)
print("* Validation set:",  X_val.shape, y_val.shape)
print("* Test set:",   X_test.shape, y_test.shape)

## EDA

In [ ]:
# Define class names. CIFER 100 labels
class_names = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle',
    'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
    'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard',
    'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain',
    'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree',
    'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket',
    'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider',
    'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor',
    'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]

In [ ]:
# Create a DataFrame for label frequency distribution
df_freq = pd.DataFrame(columns=['Set', 'Label', 'Frequency'])

In [ ]:
def count_labels(dataset, dataset_name):
    """
    Helper function to count occurrences of each label and print them
    """
    global df_freq
    unique, counts = np.unique(dataset, return_counts=True)  # Get label frequencies
    for label, frequency in zip(unique, counts):
        df_freq = pd.concat([df_freq, pd.DataFrame([{'Set': dataset_name, 'Label': class_names[label], 'Frequency': frequency}])], ignore_index=True)
        print(f"* {dataset_name} - {class_names[label]}: {frequency} images")  # Print formatted output

In [ ]:
count_labels(y_train, "Train")
count_labels(y_test, "Test")
count_labels(y_val, "Validation")

In [ ]:
# Visualize the label distribution and save image
sns.set_style("whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.xticks(rotation=45)
plt.title("Label Frequency Distribution in Train, Validation, and Test Sets")
plt.show()

In [ ]:
# Current data shape:
X_train.shape

In [ ]:
# Reshape CIFER 100 data for CNN
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Check the new shape
print(X_train.shape)  # Expected output: (48000, 28, 28, 1)

In [ ]:
X_train.max()

In [ ]:
# **Convert labels to categorical format**
n_labels = 100  # CIFER 100 classes
y_train = to_categorical(y_train, num_classes=n_labels)
y_val = to_categorical(y_val, num_classes=n_labels)
y_test = to_categorical(y_test, num_classes=n_labels)

In [ ]:
y_test

## Building model

In [ ]:
# Model 1: Baseline CNN (Improved for First-Class Standards)
def build_tf_model(input_shape, n_labels):
  model = Sequential()

  # First Convolutional Block
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
  model.add(MaxPool2D(pool_size=(2, 2)))

  # Second Convolutional Block
  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Flatten())

  # Dense Layers
  model.add(Dense(256, activation='relu'))  # Increased neurons
  model.add(Dropout(0.4))                   # Increased dropout for regularization

  # Output Layer
  model.add(Dense(n_labels, activation='softmax'))

  # Compile with lower learning rate for better convergence
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                metrics=['accuracy'])

  return model


In [ ]:
# Model 2: Deeper CNN with Batch Normalization
def build_tf_model_v2(input_shape, n_labels):
  model = Sequential()

  # First Conv Block
  model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2)))

  # Second Conv Block
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2)))

  model.add(Flatten())

  # Dense Layers
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))

  # Output Layer
  model.add(Dense(n_labels, activation='softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                metrics=['accuracy'])
  
  return model


In [ ]:
# Model 3: Compact CNN with Larger Kernels
def build_tf_model_v3(input_shape, n_labels):
  model = Sequential()

  # First Conv Layer with 5x5 kernel
  model.add(Conv2D(64, (5, 5), activation='relu', padding='same', input_shape=input_shape))
  model.add(MaxPool2D(pool_size=(2, 2)))

  # Second Conv Layer with 5x5 kernel
  model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Flatten())

  # Dense Layers
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))

  # Output Layer
  model.add(Dense(n_labels, activation='softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
                metrics=['accuracy'])

  return model


In [ ]:
model1 = build_tf_model(X_train.shape[1:], n_labels)
model2 = build_tf_model_v2(X_train.shape[1:], n_labels)
model3 = build_tf_model_v3(X_train.shape[1:], n_labels)

## Fit the model

In [ ]:
# Early stopping callback (shared across all models)
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)

# === Model 1 Training ===
model1 = build_tf_model(input_shape=X_train.shape[1:], n_labels=n_labels)
history1 = model1.fit(x=X_train,
                      y=y_train,
                      epochs=4,
                      validation_data=(X_val, y_val),
                      verbose=1,
                      callbacks=[early_stop])

# === Model 2 Training ===
model2 = build_tf_model_v2(input_shape=X_train.shape[1:], n_labels=n_labels)
history2 = model2.fit(x=X_train,
                      y=y_train,
                      epochs=4,
                      validation_data=(X_val, y_val),
                      verbose=1,
                      callbacks=[early_stop])

# === Model 3 Training ===
model3 = build_tf_model_v3(input_shape=X_train.shape[1:], n_labels=n_labels)
history3 = model3.fit(x=X_train,
                      y=y_train,
                      epochs=4,
                      validation_data=(X_val, y_val),
                      verbose=1,
                      callbacks=[early_stop])


## Model evaluation

In [ ]:
# Convert training histories to DataFrames
history1_df = pd.DataFrame(history1.history)
history2_df = pd.DataFrame(history2.history)
history3_df = pd.DataFrame(history3.history)

# Add model name column to each for easy comparison
history1_df['model'] = 'Model 1'
history2_df['model'] = 'Model 2'
history3_df['model'] = 'Model 3'

# Combine all histories into one DataFrame
full_history = pd.concat([history1_df, history2_df, history3_df], ignore_index=True)
full_history.head()


## Plot accuracy and loss

In [ ]:
# Plot Loss for All Models
sns.set_style("whitegrid")

plt.figure(figsize=(10, 5))
plt.plot(history1_df['loss'], '.-', label='Model 1 - Train Loss')
plt.plot(history1_df['val_loss'], '.-', label='Model 1 - Val Loss')
plt.plot(history2_df['loss'], '.-', label='Model 2 - Train Loss')
plt.plot(history2_df['val_loss'], '.-', label='Model 2 - Val Loss')
plt.plot(history3_df['loss'], '.-', label='Model 3 - Train Loss')
plt.plot(history3_df['val_loss'], '.-', label='Model 3 - Val Loss')
plt.title("Loss Comparison")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

print("\n")

# Plot Accuracy for All Models
plt.figure(figsize=(10, 5))
plt.plot(history1_df['accuracy'], '.-', label='Model 1 - Train Acc')
plt.plot(history1_df['val_accuracy'], '.-', label='Model 1 - Val Acc')
plt.plot(history2_df['accuracy'], '.-', label='Model 2 - Train Acc')
plt.plot(history2_df['val_accuracy'], '.-', label='Model 2 - Val Acc')
plt.plot(history3_df['accuracy'], '.-', label='Model 3 - Train Acc')
plt.plot(history3_df['val_accuracy'], '.-', label='Model 3 - Val Acc')
plt.title("Accuracy Comparison")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
# Evaluate all models on the test set
test_loss1, test_acc1 = model1.evaluate(X_test, y_test, verbose=0)
test_loss2, test_acc2 = model2.evaluate(X_test, y_test, verbose=0)
test_loss3, test_acc3 = model3.evaluate(X_test, y_test, verbose=0)

# Display results
print(f"Model 1 - Test Accuracy: {test_acc1:.4f}, Loss: {test_loss1:.4f}")
print(f"Model 2 - Test Accuracy: {test_acc2:.4f}, Loss: {test_loss2:.4f}")
print(f"Model 3 - Test Accuracy: {test_acc3:.4f}, Loss: {test_loss3:.4f}")


In [ ]:
def confusion_matrix_and_report(X, y, pipeline, label_map):
  """
  Prints the confusion matrix and classification report.
  Assumes one-hot encoded y and a trained model (pipeline).
  """
  # Make predictions (probability vectors)
  prediction = pipeline.predict(X)
  prediction = np.argmax(prediction, axis=1)
  y_true = np.argmax(y, axis=1)

  print('---  Confusion Matrix  ---')
  cm = confusion_matrix(y_true=y_true, y_pred=prediction)
  cm_df = pd.DataFrame(cm,
                       columns=[f"Predicted: {label}" for label in label_map],
                       index=[f"Actual: {label}" for label in label_map])
  print(cm_df)
  print("\n")

  print('---  Classification Report  ---')
  print(classification_report(y_true, prediction, target_names=label_map, zero_division=0))


In [ ]:
def clf_performance(X_train, y_train, X_test, y_test, X_val, y_val, pipeline, label_map):
    """
    Prints classification performance (confusion matrix & report)
    for Train, Validation, and Test sets using a trained model.
    """
    print("\n" + "="*30)
    print("#### Train Set ####")
    print("="*30 + "\n")
    confusion_matrix_and_report(X_train, y_train, pipeline, label_map)

    print("\n" + "="*30)
    print("#### Validation Set ####")
    print("="*30 + "\n")
    confusion_matrix_and_report(X_val, y_val, pipeline, label_map)

    print("\n" + "="*30)
    print("#### Test Set ####")
    print("="*30 + "\n")
    confusion_matrix_and_report(X_test, y_test, pipeline, label_map)


In [ ]:
# Classification performance for Model 1
clf_performance(X_train, y_train,
                X_test, y_test,
                X_val, y_val,
                model1,
                label_map=class_names)

# Classification performance for Model 2
clf_performance(X_train, y_train,
                X_test, y_test,
                X_val, y_val,
                model2,
                label_map=class_names)

# Classification performance for Model 3
clf_performance(X_train, y_train,
                X_test, y_test,
                X_val, y_val,
                model3,
                label_map=class_names)


## Prediction

In [ ]:
index = 102
my_garment = X_test[index]
class_index = np.argmax(y_test[index])
print("Image shape:", my_garment.shape)
print("One-hot label:", y_test[index])
print(f"This is '{class_names[class_index]}'")

sns.set_style('white')
plt.imshow(my_garment)  # No reshape needed; it's already (32, 32, 3)
plt.title(f"Class: {class_names[class_index]}")
plt.axis('off')
plt.show()

In [ ]:
my_garment.shape

In [ ]:

live_data = np.expand_dims(my_garment, axis=0)
print(live_data.shape)

In [ ]:
prediction_proba = model.predict(live_data)
prediction_proba

In [ ]:
prediction_class = np.argmax(prediction_proba, axis=1)
prediction_class

In [ ]:
# create an empty dataframe, that will show the probability per class
# we set the probabilities as the prediction_proba
prob_per_class= pd.DataFrame(data=prediction_proba[0],
                             columns=['Probability']
                             )

# we round the values to 3 decimal points, for better visualization
prob_per_class = prob_per_class.round(3)

# we add a column to prob_per_class that shows the meaning of each class
# in this case, the species name that is mapped in the target_classes
prob_per_class['Results'] = class_names

prob_per_class

## Plot prediction probability for each garment in the dataset

In [ ]:
fig = px.bar(
        prob_per_class,
        x = 'Results',
        y = 'Probability',
        range_y=[0,1],
        width=600, height=400,template='seaborn')
fig.update_xaxes(type='category')
fig.show()